# Code preparation

In [1]:
# colab_LDA/NMF_runner v1.0

# fit LDA/NMF with crisis_12
# Next: Add 20news, add run with all the configuration, add TC/TD

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install contractions
!pip install octis
!pip install bertopic
!pip install top2vec
!pip install scikit_learn==1.0.2
!pip install datashader
!pip install symspellpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev

In [4]:
%cd /content/drive/MyDrive/SS_2022_Praktikum

/content/drive/MyDrive/SS_2022_Praktikum


In [5]:
import os
import sys
import importlib
import nltk
import pandas as pd

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
# Should I in src?
assert os.path.exists('src'), f"[ERROR] The path src not detected in the parent directory '{os.getcwd()}'."

if os.getcwd().endswith('/notebooks'):
    os.chdir('..')
    sys.path.append('./src')

print(f'[INFO] Current Directory: "{os.getcwd()}".')

[INFO] Current Directory: "/content/drive/MyDrive/SS_2022_Praktikum".


# Training

In [7]:
configs = {
    'dataset': 'crisis_07',
    'preprocessing_funcs': [
        'to_lowercase',
        'standardize_accented_chars',
        'remove_url',
        'expand_contractions',
        'remove_mentions',
        'remove_hashtags',
        # 'remove_new_lines',
        'keep_only_alphabet',
        # 'remove_extra_spaces',
        'remove_english_stop_words',
        'lemmatize_noun'
    ],
    'algorithm': 'lda',
    'algorithm_args': {
        'num_topics': 2,
        'model_name': 'lda',     # Specific for LDA/NMF
        'data_save_dir': './OCTIS_data_tmp',   # Specific for LDA/NMF
        'random_state': 100,    # Specific for LDA/NMF
        # 'embedding_model': 'doc2vec',
        # 'embedding_model': 'universal-sentence-encoder',
        # 'embedding_model': 'universal-sentence-encoder-large', # WORKS VERY WELL
        # 'embedding_model': 'distiluse-base-multilingual-cased',
        #'doc2vec_speed': 'learn',
        #'min_count': 50,
        #'umap_args': {
        #    'n_neighbors': 15,
        #    'n_components': 5,
        #    'metric': 'cosine',
        #    'random_state': 42  # Try to always include this for reproducibility, github.com/ddangelov/Top2Vec/issues/86
        },
        #'hdbscan_args': {
        #    'min_cluster_size': 15,
        #    'metric': 'euclidean',
        #    'cluster_selection_method': 'eom'
        #}
    }

In [8]:
from src import preprocessor
from src.utils import load_documents

docs,labels = load_documents(dataset=configs['dataset'])
if 'preprocessing_funcs' in configs:
    docs = preprocessor.run(data=docs, prep_functions=configs['preprocessing_funcs'])
    
algorithm_args = configs['algorithm_args']
algorithm_args.update(data_name=configs['dataset'],docs=docs,labels=labels)

algorithm_name = configs['algorithm'].lower()
if algorithm_name == 'lda' or 'nmf':
    from src import LDA_NMF_runner
    df_output_doc_topic, df_output_topic_word = LDA_NMF_runner.runner(args=algorithm_args)

[INFO] Available Preprocessing Functions in the Module:['to_lowercase', 'standardize_accented_chars', 'remove_url', 'expand_contractions', 'expand_missing_delimiter', 'remove_mentions', 'remove_hashtags', 'keep_only_alphabet', 'remove_new_lines', 'remove_extra_spaces', 'remove_html_tags', 'remove_english_stop_words', 'lemmatize', 'lemmatize_verb', 'lemmatize_noun', 'lemmatize_adjective', 'correct_typo']
[INFO] Preprocessing starting..
[INFO] These string preprocessing methods will be applied to the data in order:
(  'to_lowercase',
   'standardize_accented_chars',
   'remove_url',
   'expand_contractions',
   'remove_mentions',
   'remove_hashtags',
   'keep_only_alphabet')
[INFO] Then, these tokenized preprocessing methods will be applied to the data in order:
(  '__tokenize',
   'remove_english_stop_words',
   'lemmatize_noun',
   '__glue')
[INFO] Preprocessing completed in 5.039 seconds..


2022-07-19 20:41:34,559 WARNING:too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy


In [9]:
# See doc_topic_matrix
df_output_doc_topic

,run_id,Document ID,Document,Real Label,Assigned Topic Num,Assignment Score
0,1658263294,0,rt one region supporting queensland flood appe...,flood,0,0.870156
1,1658263294,1,rt villager district desperate help many broken,earthquake,1,0.918917
2,1658263294,2,rt provide clean drinking water family please ...,earthquake,0,0.867119
3,1658263294,3,state flood crisis start ease sky news austral...,flood,0,0.655955
4,1658263294,4,queensland flood crisis deepens death toll ris...,flood,0,0.959542
...,...,...,...,...,...,...
10216,1658263294,10216,rt iaf today evacuated casualty amp person cas...,earthquake,1,0.947671
10217,1658263294,10217,hi bay could give follow back speak nepal amp ...,earthquake,1,0.956352
10218,1658263294,10218,queensland flood crisis deepens australian,flood,0,0.914386
10219,1658263294,10219,latest news queensland flood crisis deepens au...,flood,0,0.932758


In [10]:
# See Topic_Word_matrix
df_output_topic_word

,run_id,method,method_specific_params,dataset,num_given_topics,reduced,topic_num,topic_size,topic_words,word_scores,num_detected_topics,num_final_topics,duration_secs
0,1658263294,lda,None,crisis_07,2,None,0,4539,"[flood, queensland, rt, crisis, australia, ris...","[0.3243813, 0.22452883, 0.115964554, 0.0769460...",None,None,7.73
1,1658263294,lda,None,crisis_07,2,None,1,5682,"[nepal, rt, flood, australia, earthquake, peop...","[0.2629625, 0.18607771, 0.12459937, 0.10106654...",None,None,7.73


# Visualization

In [11]:
import importlib
from src import visualizer; importlib.reload(visualizer)

<module 'src.visualizer' from '/content/drive/MyDrive/SS_2022_Praktikum/src/visualizer.py'>

In [12]:
visualizer.visualize_labels_per_topic(df_output_doc_topic = df_output_doc_topic,
                                      df_output_topic_word = df_output_topic_word,
                                      top_n_topics = algorithm_args['num_topics'])

In [13]:
visualizer.visualize_barchart(df_output_topic_word = df_output_topic_word,n_words = 5)

In [14]:
# First Documents
for i in sorted(df_output_topic_word['topic_num'].to_list()):
    x = df_output_doc_topic.query(f'`Assigned Topic Num` == {i}').sort_values(by ='Assignment Score',ascending=False).drop('run_id',axis=1).head()
    with pd.option_context('display.max_colwidth', None):
        display(x)

,Document ID,Document,Real Label,Assigned Topic Num,Assignment Score
5588,5588,rt abbott raise spectre new flood levy tony abbott warned labor may seek raise tax fund queensland http,flood,0,0.972622
6307,6307,rt abbott raise spectre new flood levy tony abbott warned labor may seek raise tax fund queensland http,flood,0,0.972622
8679,8679,rt abbott raise spectre new flood levy tony abbott warned labor may seek raise tax fund queensland ht,flood,0,0.972246
5182,5182,rt abbott raise spectre new flood levy tony abbott warned labor may seek raise tax fund queensland ht,flood,0,0.972245
6290,6290,rt abbott raise spectre new flood levy tony abbott warned labor may seek raise tax fund queensland ht,flood,0,0.972244


,Document ID,Document,Real Label,Assigned Topic Num,Assignment Score
97,97,rt famous nepalese aea ae ae ea aea ea ae ae ea ae ea ae die country aa,earthquake,1,0.972684
8194,8194,rt ada ada ada ad report team heading road nepal join rescue amp relief team reportedly reaching later tonight amp,earthquake,1,0.970910
10194,10194,google exec dy mt everest avalanche nepal quake google executive dan fredinburg among lost life due,earthquake,1,0.969754
2148,2148,quake aid need acute nepal capital village quake aid need acute nepal capital village near epicenter,earthquake,1,0.969092
9364,9364,nepal quake survivor drank urine trapped hour nepal quake survivor drank urine trapped rubble hour,earthquake,1,0.968288


In [15]:
assert False

AssertionError: ignored

In [ ]:
a = [0.2, 0.3]
import numpy as np
a /= 0.5
a

In [ ]:
from gensim.corpora import Dictionary

documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

dataset = []
for text in documents:
    tokens = nltk.word_tokenize(text)
    dataset.append(tokens)
vocab = Dictionary(documents=dataset, prune_at=20000)

In [ ]:
vocab.filter_extremes(no_below=1, no_above=1, keep_n=10) 

In [ ]:
word_list = []
for key in vocab.token2id.keys():
    word_list.append(key)
word_list

In [ ]:
# Then vocabulary.txt is obtained using the function we have

In [ ]:
for i in range(len(documents)):
    new_sen = []
    for word in nltk.word_tokenize(documents[i]):
        if word in word_list:
            new_sen.append(word)
    documents[i] = ' '.join(new_sen).strip()

In [ ]:
documents

In [ ]:
vocab.token2id

In [ ]:
vocab.id2token

In [ ]:
vocab.cfs

In [ ]:
vocab.filter_extremes(no_below=2)

# Archive

In [ ]:
assert False

In [ ]:
# In Archive
from src import LDA_NMF_runner
importlib.reload(LDA_NMF_runner)

# Please choose the entries from the following sets:

# model: ['LDA', 'NMF']
# dataset: ['crisis_12', '20news']

# Other parameters
# top_n_topics: int (This decides how many topics are shown)
# n_words: int (This decides how many words are shown)

settings = {'model': 'NMF',
            'dataset': 'crisis_12',
            'top_n_topics': 5,
            'n_words': 5}

save_dir_crisis_12 = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12'

Doc_Topic_df, Topic_Word_df = LDA_NMF_runner.runner(model_name = settings['model'],
                                                    dataset_name = settings['dataset'],
                                                    top_n_topics = settings['top_n_topics'],
                                                    n_words = settings['n_words'],
                                                    save_dir = save_dir_crisis_12,
                                                    random_state = 100)